In [12]:
from flask import Flask, request,jsonify, Response

import joblib
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb
import numpy as np
import pandas as pd
import io
import csv
import json

app = Flask(__name__)

# 保存された正規化モデルを読み込む
scaler = joblib.load('scaler.pkl')

# 保存されたPCAモデルを読み込む
pca_model = joblib.load('pca_model100.pkl')

model = lgb.Booster(model_file='lgbmodel')

# CSVデータを読み込む
df = pd.read_csv("test6.txt", sep='\t', encoding='utf-16')#ここのファイル名を変更してください
tags=df[["Mark","Company name"]]
df = df.applymap(lambda x: str(x).replace(',', ''))  # カンマを削除

df=df.drop(["Mark","Company name"],axis=1)

    
df=df.replace("n.a.",np.float32(0.0))
df=df.replace(",","")

df=df.astype("float32")


    
# データを正規化する
standalized_df = pd.DataFrame(scaler.transform(df),columns=df.columns)

    

# データをPCAモデルに適用して、次元を削減する
pca_data = pca_model.transform(standalized_df)
pca_df = pd.DataFrame(pca_data)


# モデルを使用して予測を行う
predictions = model.predict(pca_df.values)
    
predicted=pd.concat([tags,pd.DataFrame(predictions)],axis=1)


predicted    

,Mark,Company name,0
0,1,JAPAN REAL ESTATE INVESTMENT CORP,13.757251


In [13]:
predicted.to_csv("predicted.csv")